In [ ]:
! apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,093 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
! pip install Pillow
! pip install pytesseract

In [10]:
import pytesseract
from PIL import ImageEnhance, ImageFilter, Image
import spacy
from pytesseract import image_to_string
import json
import os
import re
import cv2
import numpy as np

#import tesseract

In [44]:
#Fonction pour convertir l'image en text
def convert_image_to_text(img):
    print("Conversion de l'image en texte...")
    text = image_to_string(img)
     # Supprimer les espaces multiples et les sauts de ligne
    clean_text_1 = re.sub(r'\s+', ' ', text)

    # Supprimer les caractères spéciaux indésirables
    clean_text = re.sub(r'[^\w\s\.,:\-\+\(\)\[\]\{\}\?\!\;\"]', '', clean_text_1)
    return clean_text.strip()



In [38]:
def save_text_to_json(img):
    extracted_text = convert_image_to_text(img)
    filename = os.path.splitext(img)[0] + ".json"
    json_data = {
        "filename": img,
        "text": extracted_text
    }
    output_json_path = os.path.join(filename)
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)
    print(f"Le texte extrait de {img} a été sauvegardé dans {output_json_path}")

In [ ]:
save_text_to_json('/content/Inputs/mariage1.jpg')

Conversion de l'image en texte...
Le texte extrait de /content/Inputs/mariage1.jpg a été sauvegardé dans /content/Inputs/mariage1.json


In [45]:
convert_image_to_text('/content/Inputs/passeport3.jpg')

Conversion de l'image en texte...


'ideonité. sagnature et cachet de Fautorite Passeport Type () Code pays (2) WT Pasieport (3) Pp coG GA0240649 Nom (4) MPINI Présoms (5) ADIREL GARLEY EDNICK Date de nalssance (6) Leu de naissarce (77 13. SUILJUL.97 POINTE-NOIR  Natonate 87 Seve (9) f: CONGOLAISE M Protession (10) ETUDIANT Dote détablissement (22) Leu Sémission (13) 4 30 NOVNOV 16 BRAZZAVILLE. Date Sexpiration (12) 29 NOVNOV 21 COGMPINIADIRELGARLEYEDNICK 10 2406497C0G9707139M211129004'

In [ ]:
from spacy import displacy

from spacy

In [ ]:
!python -m spacy download fr_core_news_md

In [25]:
#NER with Spacy

def extract_entities(brute_file):
    text = convert_image_to_text(brute_file)
    nlp = spacy.load("fr_core_news_md")
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [26]:
extract_entities('/content/Inputs/mariage1.jpg')

Conversion de l'image en texte...


[('CONGO PROVINCE DU', 'MISC'),
 ('ACTE DE MARIAGE', 'MISC'),
 ('CINQUIEME', 'MISC'),
 ('NOVEMBRE', 'ORG'),
 ('FRANK', 'ORG'),
 ('KAFYOTO', 'ORG'),
 ('Officier', 'ORG'),
 ('Bourgmestre de la Commune de Manika', 'MISC'),
 ('Kolwezi', 'LOC'),
 ('Province duu Lualaba', 'LOC'),
 ('REPUBLIQUE DEMOCRATIQUE DU CONGO', 'MISC'),
 ('FARAI CHIRODZERO', 'LOC'),
 ('ZIMBABWE', 'ORG'),
 ('DIX-HUITIEME', 'MISC'),
 ('MARS', 'PER'),
 ('Ian MIL', 'PER'),
 ('QUATRE-VINGT ET UN', 'MISC'),
 ('Monsieur PARADZAI T.CHIRODZERO', 'PER'),
 ('Madame ENIA', 'PER'),
 ('Profession', 'LOC'),
 ('REPUBLIQUE DEMOCRATIQUE DU CONGO', 'MISC'),
 ('DEUX CENT QUATRE-VINGT-DIX-SEPT', 'MISC'),
 ('Vavenue UNGU-NGANDU', 'PER'),
 ('INDUSTRIEL', 'MISC'),
 ('MANIKA', 'MISC'),
 ('KOLWEZI', 'MISC'),
 ('Mademoiselle KAMUZE', 'MISC'),
 ('KIBAWA', 'ORG'),
 ('PATRICIA', 'ORG'),
 ('KOLWEZI', 'MISC'),
 ('ONZIEME', 'MISC'),
 ('SEPTEMBRE', 'ORG'),
 ('Ian MIL', 'PER'),
 ('CENT QUATRE-VINGT-DEUX', 'MISC'),
 ('Monsieur', 'PER'),
 ('MPANGA KAMUZE'

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json

# Charger le modèle et le tokenizer LayoutLMv3
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv3-base")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/layoutlmv3-base")

# Fonction pour préparer l'entrée
def prepare_input(img):
    inputs = tokenizer(image_to_string(img), return_tensors="pt", truncation=True, max_length=512)
    return inputs["input_ids"], inputs["attention_mask"]

# Fonction pour extraire les informations
def extract_marriage_certificate_info(text):
    input_ids, attention_mask = prepare_input(text)

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = outputs.logits.argmax(-1)[0]

    # Interprétation des résultats (à compléter selon vos besoins spécifiques)
    interpretation = interpret_predictions(predictions)

    return interpretation

# Fonction d'interprétation des prédictions (à adapter selon vos besoins)
def interpret_predictions(predictions):
    # Implémentez ici la logique d'interprétation des prédictions
    # Par exemple, vous pourriez créer un dictionnaire de correspondance entre les prédictions et les informations extraites
    interpretation = {}

    # Exemple simplifié :
    interpretation["informationsEpoux"] = {"nom": "", "prenom": ""}
    interpretation["informationsEpouse"] = {"nom": "", "prenom": ""}

    # Ajoutez plus d'informations selon votre besoin

    return interpretation



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
extract_marriage_certificate_info('/content/Inputs/mariage1.jpg')

ValueError: Words must be of type `List[str]` (single pretokenized example), or `List[List[str]]` (batch of pretokenized examples).